In [1]:
import pybullet_data
import pybullet as p
import pybullet_industrial as pi
import os
import time
import numpy as np

p.connect(p.GUI)  # Use p.DIRECT for headless
#p.connect(p.DIRECT)
data_path = pybullet_data.getDataPath()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
print(pybullet_data.getDataPath())
plane_id = p.loadURDF("plane.urdf")

# Get the path to the PyBullet data directory
data_path = pybullet_data.getDataPath()

# List the files in the PyBullet data directory
files = os.getcwd()
dirname = os.path.join(files,'robots')
assets = os.path.join(files,'assets')

urdf_file1 = os.path.join(dirname,'rb5_850e.urdf')
asset1 = os.path.join(assets,'specimen.urdf')
#endeffector = os.path.join(assets,'endeffector.urdf')
endeffector = os.path.join(assets,
                                'gripper_cylinder.urdf')

start_orientation = p.getQuaternionFromEuler([0, 0, 0])

cube1 = p.loadURDF("cube.urdf", np.array(
        [0, 0, 0.5]), useFixedBase=True)
robot = pi.RobotBase(urdf_file1, np.array(
        [0, 0, 1]), start_orientation)
cube2 = p.loadURDF("cube.urdf", np.array(
        [1, 0, 0.5]), useFixedBase=True)
specimen = p.loadURDF(asset1, np.array(
       [1.0, 0, 1.3]), useFixedBase=True)
meshScale = [1, 1, 1]

pi.draw_robot_frames(robot, life_time=0)

tool = pi.EndeffectorTool(endeffector, np.array([2.0, 0, 1.5]), start_orientation, robot)
#tool.couple(robot, 'tcp')

#camera = pi.Camera([2, 1, 1.5], [0, 0, 0], [0, 0, 1], 0.5, 0.5, 640, 480)

# Set end-effector
# tool_properties = {'maximum distance': 0.1,
#                         'opening angle': 0,
#                         'material': pi.Plastic,
#                         'material properties': {'particle size':0.002,
#                                                 'color' : [1, 1, 0, 1]},
#                         'number of rays': 1}

# tool = pi.Extruder(
#     endeffector, [2.0, 0, 1.5], start_orientation, tool_properties)
# 

# Set gravity
p.setGravity(0, 0, -9.81)

#extruder.set_tool_pose([0,0,0], p.getQuaternionFromEuler([0, 0, 0]))

# Simulate for 10 seconds

#for i in range(240 * 20):  # 240 Hz simulation frequency
#    p.stepSimulation()
#    time.sleep(1./240.)
urdf_file3 = os.path.join(assets, 'camera.urdf')
camera_parameters = {'width': 480, 'height': 240, 'fov': 60,
                         'aspect ratio': 1, 'near plane distance': 0.01, 'far plane distance': 2}
camera_orientation = p.getQuaternionFromEuler([1.57, 0, 1.57])
camera_position = tool.get_tool_pose()[0] +  np.array([0, 0, 0])
camera = pi.Camera(urdf_file3, camera_position,
                    camera_orientation, camera_parameters)

c:\Users\b34b3\AppData\Local\Programs\Python\Python312\Lib\site-packages\pybullet_data


In [24]:
def check_collision(visualize=False):
    contacts = p.getContactPoints() 
    contacts = [c for c in contacts if c[1] >= 4] # 로봇의 베이스나, 엔드이펙터와 플랜지사이의 충돌은 무시
    if contacts:
        print("TCP 링크가 무언가와 충돌 발생!")
        for contact in contacts:
            print(contact)
    
    if visualize:
        for c in contacts:
            pos_onB = c[6]
            p.removeAllUserDebugItems()
            p.addUserDebugPoints(
                pointPositions=[pos_onB],
                pointColorsRGB=[[1, 0, 0]],   # 빨간색
                pointSize=50,
                lifeTime=10
            )

def update_camera(camera_position, camera_orientation):
    camera_id = None
    for body_id in range(p.getNumBodies()):
        body_name = p.getBodyInfo(body_id)[1].decode()
        if "camera" in body_name:  # 카메라 URDF 파일명이 포함된 경우
            camera_id = body_id
            break
    pos_offset = np.array([0.1, 0, 0.04])
    ori_offset = np.array([0.5, 0.5, 0, 0])
    p.resetBasePositionAndOrientation(camera_id, camera_position + pos_offset, camera_orientation + ori_offset)

def update_specimen(specimen_position, specimen_orientation):
    specimen_id = None
    for body_id in range(p.getNumBodies()):
        body_name = p.getBodyInfo(body_id)[1].decode()
        if "my_specimen" in body_name:  # my_specimen URDF 파일명이 포함된 경우
            specimen_id = body_id
            break
    p.resetBasePositionAndOrientation(specimen_id, specimen_position, specimen_orientation)

base_pose = np.array([0, 0, 0.5])
set_pose = np.array([0.860, 0, 0.81])
base_orientation = np.array([1.57, 0, 1.57])

target_pose = base_pose + set_pose


camera_orientation = p.getQuaternionFromEuler([1.57, 0, 1.57])
camera_position = tool.get_tool_pose()[0] +  np.array([0, 0, 0])


for _ in range(60 * 1):  # 240 Hz simulation frequency
    tool.set_tool_pose(target_pose, p.getQuaternionFromEuler(base_orientation))
    img = camera.get_image()
    p.stepSimulation()
    camera_orientation = p.getQuaternionFromEuler([1.57, 0, 1.57])
    camera_position = tool.get_tool_pose()[0] +  np.array([0, 0, 0])
    update_camera(camera_position, camera_orientation)
    time.sleep(1./60.)

check_collision(visualize=True)

In [15]:
specimen_position = np.array([1.0, 0, 1.30])
specimen_orientation = p.getQuaternionFromEuler([0.2, 0, 0])
update_specimen(specimen_position, specimen_orientation)
p.stepSimulation()


()

In [37]:
def show_point(tool_pos):
    p.addUserDebugPoints(
    pointPositions=[tool_pos],
    pointColorsRGB=[[1, 0, 0]],  # Red
    pointSize=50,
    lifeTime=1
)
show_point(tool.get_tool_pose()[0]+np.array([0.15,0,0]))
#tool.get_tool_pose()[0]

# visualize the pose

In [23]:
p.stepSimulation()

()

In [4]:
# img show
import cv2
img_uint8 = img.astype(np.uint8)
rgb = img_uint8[:,:,:3]
bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
cv2.imshow('image', bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
urdf_file3 = os.path.join(assets, 'camera.urdf')
camera_parameters = {'width': 480, 'height': 240, 'fov': 60,
                         'aspect ratio': 1, 'near plane distance': 0.01, 'far plane distance': 2}
camera_orientation = p.getQuaternionFromEuler([1.57, 0, 1.57])
camera_position = tool.get_tool_pose()[0] +  np.array([0, 0, 0])
camera = pi.Camera(urdf_file3, camera_position,
                    camera_orientation, camera_parameters)
img = camera.get_image()

In [3]:
camera_id = None
for body_id in range(p.getNumBodies()):
    body_name = p.getBodyInfo(body_id)[1].decode()
    if "camera" in body_name:  # 카메라 URDF 파일명이 포함된 경우
        camera_id = body_id
        break

print("찾은 카메라의 Body ID:", camera_id)

찾은 카메라의 Body ID: 6


In [8]:
for body_id in range(p.getNumBodies()):
    body_info = p.getBodyInfo(body_id)  # 바디 정보 가져오기
    print("Body ID:", body_id, "Body Name:", body_info[1].decode())  # UTF-8 변환

Body ID: 0 Body Name: plane
Body ID: 1 Body Name: cube
Body ID: 2 Body Name: rb5_850e
Body ID: 3 Body Name: cube
Body ID: 4 Body Name: my_specimen
Body ID: 5 Body Name: gripper
Body ID: 6 Body Name: camera


In [4]:
pos_offset = np.array([0.1, 0, 0.04])
ori_offset = np.array([0.5, 0.5, 0, 0])
p.resetBasePositionAndOrientation(camera_id, camera_position + pos_offset, camera_orientation + ori_offset)
img = camera.get_image()

In [9]:
check_collision(visualize=True)

TCP 링크가 무언가와 충돌 발생!
(0, 4, 5, -1, -1, (0.9958659598715782, 3.5049397318866585e-05, 1.3070350950326735), (0.9964378897260121, 2.324144713081351e-05, 1.3070422607740793), (0.9997085143105361, -0.020639783442026008, 0.012525404364211117), -0.0005720966124093917, 2454.312344588046, 54.97487557292521, (0.020641402676553497, 0.9997869435512471, 0.0), -407.0712180170591, (-0.012522735746038084, 0.0002585419151683422, 0.9999215540458728))
